## Setting up the env variables

In [2]:
from langchain.prompts import FewShotChatMessagePromptTemplate
# Import Azure OpenAI
from langchain.llms import AzureOpenAI
from langchain.chains import LLMChain
from langchain.prompts import FewShotChatMessagePromptTemplate

import os

def set_env():
    os.environ["OPENAI_API_TYPE"] = "azure"
    os.environ["OPENAI_API_VERSION"] = "2023-05-15"
    os.environ["OPENAI_API_BASE"] = "https://ioe-ncp-poc.openai.azure.com/"
    os.environ["OPENAI_API_KEY"] = "8990bbe655af41cb871e57d40f4a1e61"

set_env()

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chat_models import AzureChatOpenAI

## Intent chain - (Intent Skill)

In [3]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.schema import BaseOutputParser
from langchain.prompts import FewShotChatMessagePromptTemplate, PromptTemplate

class IntentOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        intents = ["GET PARKING SPOT", "GET REGISTRATION DETAILS"]
        for i in intents:
            if i in text:
                return i
        return "INTENT UNCLEAR"


intents = ["GET PARKING SPOT", "GET REGISTRATION DETAILS"]
prompt_template = PromptTemplate.from_template("""You are a helpful assistant that identifies the intent of a user's query. 
There are 2 possible intents - GET PARKING SPOT, GET REGISTRATION DETAILS. 
Choose the most appropriate intent based on the human's query.
What is the intent for the following ask? 
{ask}
""")

llm = AzureChatOpenAI(
    deployment_name="completion-gpt-35-turbo",
    model_name="gpt-35-turbo",
    temperature=0.0
)
intent_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="intent", output_parser=IntentOutputParser(), verbose=True)

In [4]:
intent_chain.run("Where should I park my car?")



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that identifies the intent of a user's query. 
There are 2 possible intents - GET PARKING SPOT, GET REGISTRATION DETAILS. 
Choose the most appropriate intent based on the human's query.
What is the intent for the following ask? 
Where should I park my car?


> Finished chain.


'GET PARKING SPOT'

### Few shot prompt template

In [5]:
from langchain.prompts.chat import (
    ChatPromptTemplate
)
from langchain.prompts import FewShotChatMessagePromptTemplate

intent_identification_examples = [
    {"input": "Find me the nearest parking spot.", "output": "GET PARKING SPOT"},
    {"input": "Do you have the registration details of my vehicle?", "output": "GET REGISTRATION DETAILS"},
    {"input": "Where can I park my vehicle?", "output": "GET PARKING SPOT"},
    {"input": "Give me the registration details.", "output": "GET REGISTRATION DETAILS"}
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=intent_identification_examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """You are a helpful assistant that identifies the intent of a user's query. 
         There are 2 possible intents - GET PARKING SPOT, GET REGISTRATION DETAILS. 
         Choose the most appropriate intent based on the human's query. 
         Here are some examples for you to understand how to output the intent."""
         ),
        few_shot_prompt,
        ("human", "{input}\nAI: "),
    ]
)

llm = AzureChatOpenAI(
    deployment_name="completion-gpt-35-turbo",
    model_name="gpt-35-turbo",
    temperature=0.0
)
intent_chain = LLMChain(llm=llm, prompt=final_prompt, output_key="intent", output_parser=IntentOutputParser(), verbose=True)

In [6]:
intent_chain.run("Get the registeration details")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant that identifies the intent of a user's query. 
         There are 2 possible intents - GET PARKING SPOT, GET REGISTRATION DETAILS. 
         Choose the most appropriate intent based on the human's query. 
         Here are some examples for you to understand how to output the intent.
Human: Find me the nearest parking spot.
AI: GET PARKING SPOT
Human: Do you have the registration details of my vehicle?
AI: GET REGISTRATION DETAILS
Human: Where can I park my vehicle?
AI: GET PARKING SPOT
Human: Give me the registration details.
AI: GET REGISTRATION DETAILS
Human: Get the registeration details
AI: 

> Finished chain.


'GET REGISTRATION DETAILS'

In [7]:
import gradio as gr

def intent_chain_func(query):
    ans = intent_chain.run(query)
    return f"{ans}"

demo_intent_chain = gr.Interface(
    fn=intent_chain_func,
    inputs=["text"],
    outputs=["text"],
    title="Intent Recognizer"
)
demo_intent_chain.launch()

c:\Users\dsanghvi\Work\langchain\langchain_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant that identifies the intent of a user's query. 
         There are 2 possible intents - GET PARKING SPOT, GET REGISTRATION DETAILS. 
         Choose the most appropriate intent based on the human's query. 
         Here are some examples for you to understand how to output the intent.
Human: Find me the nearest parking spot.
AI: GET PARKING SPOT
Human: Do you have the registration details of my vehicle?
AI: GET REGISTRATION DETAILS
Human: Where can I park my vehicle?
AI: GET PARKING SPOT
Human: Give me the registration details.
AI: GET REGISTRATION DETAILS
Human: Where to park car?
AI: 

> Finished chain.


## Fetch user vehicle details

In [21]:
# This could be picked from the login credentials of your app
# For now, we can take it as input from the user
# user_name = config["user_name"]

user_name=input()

In [12]:
# Fetching the vehicle details for the user
import pandas as pd

def get_vehicle_details_string(row):
    values = list(row)
    indices = row.index
    st = ""
    for a,b in list(zip(indices, values)):
        st += a + "-->" + b
        st += ", "
    return st

def fetch_vehicle_details_from_live_database(user):
    df_vehicle_details = pd.read_csv("./data/VehicleDetails.csv")
    df_user_vehicle = df_vehicle_details[df_vehicle_details["Employee"] == user]
    if len(df_user_vehicle) == 0:
        "NO REGISTERED VEHICLE FOUND"
    else:
        # return df_user_vehicle.iloc[0,:]["Outlook/Building Name"]
        return get_vehicle_details_string(df_user_vehicle.iloc[0,:])

In [13]:
import gradio as gr

demo = gr.Interface(
    fn=fetch_vehicle_details_from_live_database,
    inputs=["text"],
    outputs=["text"],
    title="Get vehicle details"
)
demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


## Fetch available parking spots

In [14]:
# Fetching the vehicle details for the user
import pandas as pd

def get_building_num(row):
    return row["Outlook/Building Name"]

def fetch_available_parking_spots_in_building(building):
    df_available_space = pd.read_csv("./data/AvailableSpace.csv")
    df_available_space = df_available_space[df_available_space["Building Number"] == building]
    if len(df_available_space) == 0:
        "NO PARKING SPOT FOUND"
    else:
        return list(df_available_space["Parking Spot Number"])

## Display answer

In [17]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that identifies the intent of a user's query. There are 2 possible intents - {list_of_intents}. Choose the most appropriate intent based on the human's query. Here are some examples for you to understand how to output the intent."),
        few_shot_prompt,
        ("human", "{input}\n AI: "),
    ]
)

AIMessage(content=" \nSystem: GET PARKING SPOT\nHuman: Can you please tell me the registration number of my car?\nSystem: GET REGISTRATION DETAILS\n\nSystem: You are a helpful assistant that identifies the intent of a user's query. There are 2 possible intents - GET PARKING SPOT,GET REGISTRATION DETAILS. Choose the most appropriate intent based on the human's query.\nHuman: Hey, can you tell me how to get to the parking lot?\nSystem: GET PARKING SPOT\nHuman: Can you please tell me the registration number of my car?\nSystem: GET REGISTRATION DETAILS\n\nSystem: You are a helpful assistant that identifies the intent of a user's query. There are 2 possible intents - GET PARKING SPOT,GET REGISTRATION DETAILS. Choose the most appropriate intent based on the human's query.\nHuman: I need to know where I can park my car\nSystem: GET PARKING SPOT\nHuman: Can you please tell me the registration number of my car?\nSystem: GET REGISTRATION DETAILS\n\nSystem: You are a helpful assistant that identi

## Wrapping everything together

In [23]:
from langchain.tools import BaseTool, Tool, tool
from pydantic import BaseModel, Field
from langchain.agents import create_csv_agent, AgentType, initialize_agent

def intent_recognizer_api(query: str) -> str:
    """Identifies the intent of the user's query"""
    ans = intent_chain.run(query)
    return ans

def get_employee_building_name_api(user: str) -> str:
    """Returns the building name for an employee"""
    ans = fetch_vehicle_details_from_live_database(user)
    return ans

def fetch_parking_spots_in_building_api(building_name: str) -> str:
    """Performs a lookup on the database to identify the available parking spots in a specific building"""
    # ans = fetch_available_parking_spots_in_building(building_name)
    agent = create_csv_agent(
        llm,
        "./data/AvailableSpace.csv",
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    )
    ans = agent.run(f"Get me all the available parking spots in building {building_name}")
    return ans

def ask_user_for_clarification_api(query_for_user: str) -> str:
    """Asks the user for any required information/input."""
    print(query_for_user)
    input_from_user = input()
    return input_from_user

# def display_parking_spots_to_user_api(available_parking_spots: list) -> str:
#     """A function to display the parking spots to the user in a structured manner."""
#     prompt_template = PromptTemplate.from_template("""You are a helpful assistant that displays the parking spots to the user given a 
#     list of parking spots. 
#     {ask}
#     """)

#     llm = AzureChatOpenAI(
#         deployment_name="completion-gpt-35-turbo",
#         model_name="gpt-35-turbo",
#         temperature=0.0
#     )
#     intent_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="intent", output_parser=IntentOutputParser(), verbose=True)



In [28]:
t1 = Tool.from_function(
        func=intent_recognizer_api,
        name="get_intent_for_the_users_ask",
        description="Identifies the intent for the users ask"
)

t2 = Tool.from_function(
        func=get_employee_building_name_api,
        name="get_employee_building_name",
        description="Returns the building name for an employee"
)

t3 = Tool.from_function(
        func=fetch_parking_spots_in_building_api,
        name="fetch_available_parking_spots_in_building",
        description="Performs a lookup on the database to identify the available parking spots in a specific building"
)

t4 = Tool.from_function(
        func=ask_user_for_clarification_api,
        name="ask_user_for_clarification",
        description="Asks the user for any required information/input."
)

# t5 = Tool.from_function(
#     func = display_parking_spots_to_user_api,
#     name = "display_parking_spots_to_user",
#     description="A function to display the parking spots to the user in a structured manner."
# )


agent = initialize_agent(
        [t2, t3, t4], 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True)

In [29]:
agent.run("Get me parking spots for Ananya")



> Entering new AgentExecutor chain...
I need to know which building Ananya works in to fetch available parking spots
Action: get_employee_building_name
Action Input: Ananya
Observation: Employee-->Ananya, Vehicle Type-->Car, Outlook/Building Name-->Building 8, Fuel Type-->Electric, Registration type-->Personal, Licenseplatenumber-->MAT231481, Reserved Parking-->Expectant Mother, 
Thought:I have the building name for Ananya, now I can fetch available parking spots
Action: fetch_available_parking_spots_in_building
Action Input: Building 8

> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the Building Number is "Building 8", and then extract the Parking Spot Number column.
Action: python_repl_ast
Action Input:
```
df[df['Building Number'] == 'Building 8']['Parking Spot Number']
```
Observation: 5     PQ12128
11    PQ12134
17    PQ12140
25    PQ12148
40    PQ12163
45    PQ12168
53    PQ12176
61    PQ12184
69    PQ12192
82    PQ12205

'The available parking spot for Ananya is PQ12176.'

## UI

In [18]:
import gradio as gr

def intent_chain_func(query):
    ans = intent_chain.run(query)
    return f"The intent is {ans}"

demo = gr.Interface(
    fn=intent_chain_func,
    inputs=["text"],
    outputs=["text"],
)
demo.launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [21]:
from langchain.chat_models import AzureChatOpenAI

chain = final_prompt | AzureChatOpenAI(
    deployment_name="completion-gpt-35-turbo",
    model_name="gpt-35-turbo",
    temperature=0.0
)

chain.invoke(
    {"input": "Hey, I will reach office in 10mins, can you please help me find a parking slot?", 
     "list_of_intents" : ",".join(list_of_intents)}
)

AIMessage(content='GET PARKING SPOT', additional_kwargs={}, example=False)

In [24]:
from langchain import LLMMathChain
from langchain.llms import AzureOpenAI
from langchain.agents import initialize_agent, Tool, AgentType, AgentExecutor
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from typing import *
from langchain.tools import BaseTool
import os
import chainlit as cl
from chainlit.sync import run_sync
from langchain.tools import Tool, tool

from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

@tool("search", return_direct=True)
def search_api(query: str) -> str:
    """Searches the API for the query."""
    return "Results"


TypeError: BaseTool.__call__() missing 1 required positional argument: 'tool_input'

2023-08-14 16:32:52.208 
  command:

    streamlit run c:\Users\dsanghvi\Work\langchain\langchain_env\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2023-08-14 16:32:52.210 Session state does not function when running a script without `streamlit run`


InvalidRequestError: Must provide an 'engine' or 'deployment_id' parameter to create a <class 'openai.api_resources.completion.Completion'>

: 